In [94]:
import glob
import nbconvert
import os
import pandas as pd

In [175]:
htmlpath = "http://www.porganized.com/Scripting-Course/slides/%s"
labloc = "https://github.com/organisciak/Scripting-Course/blob/master/labs/%s"

In [176]:
# Process Slides
slides = glob.glob('slides/*.ipynb')

slides_exporter = nbconvert.SlidesExporter()
md_exporter = nbconvert.MarkdownExporter()
slides_exporter.reveal_url_prefix = '../reveal.js/'

for slide in slides:
    path, filename = os.path.split(slide)
    name, ext = os.path.splitext(filename)
    
    # Export Reveal.js slides
    slidepath = os.path.join('slides', name + '.html')
    (html, resources) = slides_exporter.from_filename(slide)
    with open(slidepath, mode='w') as f:
        f.write(html)
        
    # Save Markdown of slides, with a link to the Reveal.js version
    mdpath = os.path.join('slides', name + '.md')
    htmlslidepath = htmlpath % (name + '.html')
    (md, resources) = md_exporter.from_filename(slide)
    newmd = ("_[Link to Slides](%s)_\n\n" % htmlslidepath) + md
    
    with open(mdpath, mode='w') as f:
        f.write(newmd)

In [177]:
# Export Markdown for all labs
labs = glob.glob('labs/*.ipynb')

for lab in labs:
    path, filename = os.path.split(lab)
    name, ext = os.path.splitext(filename)
    mdpath = os.path.join('labs', name + '.md').replace(' ', '_')
    (md, resources) = md_exporter.from_filename(lab)
    labpath = labloc % (name.replace(" ", "%20") + '.ipynb')
    newmd = ("_[Link to Notebook Version of Lab](%s)_\n\n" % labpath) + md
    with open(mdpath, mode='w') as f:
        f.write(newmd)

In [181]:
allfiles = []
slidemds = glob.glob('slides/*.md')
labmds = glob.glob('labs/*.md')

def get_week(filepath):
    path, filename = os.path.split(filepath)
    weekstr = filename.split('-')[0].split('_')[0]
    return int(weekstr)

def pretty_name(filepath):
    path, filename = os.path.split(filepath)
    name, ext = os.path.splitext(filename)
    name = (name.split('-')[1]
                .strip()
                .replace('_', ' ')
                .replace('-',' ')
                .title()
           )
    return name

def format_filename(filepath):
    return filepath.replace(' ','%20').replace('\\', '/')

allfiles += [(get_week(md), 1, pretty_name(md), 'slides', format_filename(md)) for md in slidemds]
allfiles += [(get_week(md), 2, pretty_name(md), 'lab', format_filename(md)) for md in labmds]

bookfiles = (pd.DataFrame(allfiles, columns=['week', 'priority', 'pretty_name', 'type', 'filename'])
               .sort_values(['week', 'priority'])
            )
bookfiles['link'] = '[' + bookfiles.type.str.title() + ': ' + bookfiles.pretty_name + '](' + bookfiles.filename + ')'

for week, row in bookfiles.groupby('week').link:
    print("* Week %d" % week)
    links = ("   * " + row).tolist()
    print("\n".join(links))

* Week 1
   * [Slides: Introduction](slides/01-introduction.md)
   * [Lab:  Getting Started](labs/01_-_Getting_Started.md)
